<a href="https://colab.research.google.com/github/michalcohen/DataAnalysisABC/blob/main/find_invariant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installs
!pip install jupyter-dash
!pip install dash-cytoscape

In [2]:
# imports
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash
import dash_cytoscape as cyto
import urllib.request, json
import statistics

In [3]:
# Load Data
df = px.data.tips()
with urllib.request.urlopen("https://raw.githubusercontent.com/michalcohen/DataAnalysisABC/main/pass1.json") as url:
    data = json.loads(url.read().decode())

In [4]:
graph = {node["data"]["id"]: {"neighbors": list(), "inv": node["classes"].startswith("inv"), "origin": int(node["classes"].split(" ")[-1])} for node in data["nodes"]}
for edge in data["edges"]:
  graph[edge["data"]["source"]]["neighbors"].append(edge["data"]["target"])
degrees = []
for node_id, node_content in graph.items():
  node_content['degree'] = len(node_content["neighbors"])
  degrees.append(node_content['degree'])
std_deg, mean_deg, median_deg, max_deg = statistics.stdev(degrees), statistics.mean(degrees), statistics.median(degrees), max(degrees)
high_deg_nodes = [node_id for node_id, node_content in graph.items() if node_content['degree'] > mean_deg + std_deg]
high_deg_nodes[-10: -1]

['2145', '2042', '1502', '2106', '2192', '2054', '2035', '2170', '2136']

In [5]:
invariants = [node_id for node_id, node_content in graph.items() if node_content["inv"]]
len(invariants) / len(graph)



0.4405430711610487

In [6]:
group1 = [node_id for node_id, node_content in graph.items() if node_content["origin"] == 1]
len(group1) / len(graph)

0.9948501872659176

In [7]:
# Build App
app = JupyterDash(__name__)
elements = data["nodes"] + data["edges"]
app.layout = html.Div([
                       html.P("Dash Cytoscape:"),
                       cyto.Cytoscape(id='cytoscape',
                                      elements=elements,
                                      layout={'name': 'breadthfirst'},
                                      style={'width': '800px', 'height': '1000px'},
                                      stylesheet = [
        # Group selectors
        {
            'selector': 'node',
            'style': {
                'content': 'data(label)'
            }
        },
        {
            'selector': 'edge',
            'style': {
                'target-arrow-color': '#ccc',
                'target-arrow-shape': 'triangle'
            }
        },
        # Class selectors
        {
            'selector': '.inv',
            'style': {
                'background-color': 'red'
            }
        },
        {
            'selector': '.non_inv',
            'style': {
                'background-color': 'blue'
            }
        },
        {
            'selector': '.1',
            'style': {
                'line-color': 'black'
            }
        },
        {
            'selector': '.2',
            'style': {
                'line-color': 'yellow'
            }
        },
        {
            'selector': '.3',
            'style': {
                'line-color': 'green'
            }
        },
        {
            'selector': '.4',
            'style': {
                'background-color': 'purple'
            }
        }
    ]
                                      )
                       ])
app.run_server(mode='inline')

<IPython.core.display.Javascript object>